# Set-up

## Installing libraries and libcudnn8

In [ ]:
## only with GPU
import sys
sys.path.insert(0,'/kaggle/input/mi-eeg-classmeth/')
#FILEID = "1-bPsREsUCOiJHzIqi8DQrfSjTAf5VAW_"
!apt-get install --allow-change-held-packages libcudnn8=8.1.1.33-1+cuda11.2 -y
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.EEG_Tensorflow_models.git
!pip install mne
!pip install scikeras[tensorflow]
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
#!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
#!dir

In [ ]:
!pip install tensorflow-probability==0.18.* --quiet
!pip install tensorflow==2.10.* --quiet

## Import libraries

In [ ]:
# gcpds datasets
from gcpds.databases import GIGA_MI_ME
from gcpds.databases.BCI_Competition_IV import Dataset_2a
import gcpds.databases as loaddb

# freq filter 
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr

# general
import numpy as np
from scipy.signal import resample
import pickle
import warnings
import mne
from time import time
warnings.filterwarnings('ignore')

# tensorlfow 
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, AveragePooling2D, BatchNormalization, Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Layer
from tensorflow.keras.regularizers import L1L2

# scikeras
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV,StratifiedShuffleSplit
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score

## Define functions

In [ ]:
def kappa(y_true, y_pred):
    return cohen_kappa_score(np.argmax(y_true, axis = 1),np.argmax(y_pred, axis = 1))

## GIGA Science dataset

In [ ]:
def load_GIGA(db: GIGA_MI_ME,
              sbj: int,
              fs: float, 
              f_bank: np.ndarray, 
              vwt: np.ndarray, 
              new_fs: float) -> np.ndarray:

    eeg_ch_names = ['Fp1','Fpz','Fp2',
              'AF7','AF3','AFz','AF4','AF8',
              'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
              'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
              'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
              'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
              'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
              'PO7','PO3','POz','PO4','PO8',
              'O1','Oz','O2',
              'Iz']

    index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

    db.load_subject(sbj)
    X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
    X = X[:,index_eeg_chs,:] #spatial rearrangement
    X = np.squeeze(tf_repr.transform(X))
    #Resampling
    if new_fs == fs:
        print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    return X, y

## BCI2a dataset

In [ ]:
def load_BCICIV2a(db: Dataset_2a,
               sbj: int,
               fs: float, 
               f_bank: np.ndarray, 
               vwt: np.ndarray, 
               new_fs: float) -> np.ndarray:

    tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)
    # training 
    db.load_subject(sbj, mode = 'training')
    X_tr, y_tr = db.get_data() #Load all classes, all channels {EEG, EOG}, reject bad trials
    X_tr = X_tr[:,:-3,:] # pick EEG channels
    X_tr = X_tr*1e6 #uV
    X_tr = np.squeeze(tf_repr.transform(X_tr))
    # testing
    db.load_subject(sbj, mode = 'evaluation')
    X_ts, y_ts = db.get_data() #Load all classes, all channels {EEG, EOG}, reject bad trials
    X_ts = X_ts[:,:-3,:] # pick EEG channels
    X_ts = X_ts*1e6 #uV
    X_ts = np.squeeze(tf_repr.transform(X_ts))
    # merge both training and evaluation sets  
    X = np.concatenate([X_tr, X_ts], axis = 0)
    y = np.concatenate([y_tr, y_ts], axis = 0)    
    # Resampling
    if new_fs == fs:
        print('No resampling, since new sampling rate same.')
    else:
        print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
        X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    return X, y

## Define the model EEGnet

In [ ]:
from EEG_Tensorflow_models.Models import DeepConvNet, ShallowConvNet, EEGNet

# Experiment

## Experiment configuration 

In [ ]:
seed=23
folds=5
epochs_train = 500
data_set='BCICIV2a'## BCICIV2a # GIGA_MI_ME

save_folder = 'EEGnet_128Hz'
Experiment=f'{data_set}__{folds}_fold'

model_name= f'{save_folder}{epochs_train}_epoch'


In [ ]:
PATH=f'./{save_folder}/'
!mkdir '{PATH}'

## Run experiment

In [ ]:
tf.random.set_seed(seed)

if data_set=='GIGA_MI_ME':
    subjects = np.arange(52)+1
    subjects = np.delete(subjects,[28,33])
    db = GIGA_MI_ME('../input/giga-mi-me/GIGA_MI_ME/') # update path
    num_class = 2
    load_args = dict(db = db,
                fs = db.metadata['sampling_rate'],
                f_bank = np.asarray([[4., 40.]]),
                vwt = np.asarray([[2.5, 5]]),
                new_fs = 128.0)    
elif data_set=='BCICIV2a':
    subjects = np.arange(9)+1
    db = Dataset_2a('../input/bciciv2a/BCI_CIV_2a/') # update path
    num_class = 4
    load_args = dict(db = db,
                fs = db.metadata['sampling_rate'],
                f_bank = np.asarray([[4., 40.]]),
                vwt = np.asarray([[2.5, 6]]),
                new_fs = 128.0)
else:
    print('Select a valid dataset')            

t=time()
for sbj in subjects:
    load_args['sbj'] = sbj

    if data_set=='GIGA_MI_ME':
        X_train, y_train = load_GIGA(**load_args)
    elif data_set=='BCICIV2a':
        X_train, y_train = load_BCICIV2a(**load_args)

    X_train = X_train[..., np.newaxis]
    Y_train = tf.keras.utils.to_categorical(y_train,num_classes=num_class)
    # build model
    clf = KerasClassifier(
        EEGNet,
        random_state=seed,

        #model hyperparameters
        nb_classes=num_class, 
        Chans = X_train.shape[1], 
        Samples = X_train.shape[2],
        kernLength = 32,
        F1 = 8,
        D = 2,
        F2 = 16,
        norm_rate = 0.25,
        dropoutType = 'Dropout',

        #model config
        verbose=0,
        batch_size=500, #full batch        
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer="adam",
        optimizer_learning__rate=0.1,
        metrics = ['accuracy'],
        epochs = epochs_train
    )
    # search params
    param_grid =  {
                'F1':[8],
                'kernLength':[32],
                }
    
    #Gridsearch
    scoring = {"AUC": 'roc_auc', "Accuracy": make_scorer(accuracy_score),'Kappa':make_scorer(kappa)}
    
    cv = GridSearchCV(clf,param_grid,cv=StratifiedShuffleSplit(n_splits = folds, test_size = 0.2, random_state = seed),
                         verbose=0,n_jobs=1,
                         scoring=scoring,
                         refit="Accuracy",
                            )
    # frind best params with gridsearch
    cv.fit(X_train,Y_train)
    # best score
    print('Subject',sbj,'Accuracy',cv.best_score_,'elapsed time',time()-t)
    print('---------')
    
    cv.cv_results_['best_index_']=cv.best_index_
    
    #########
    cv.best_estimator_.model_.save_weights(f'{PATH}Model_{Experiment}_{model_name}_sujeto_{sbj}_'+data_set+'_4_40_weights.h5')
    with open(PATH+'Results_'+Experiment+'_'+model_name+'_sujeto_'+str(sbj)+'_'+data_set+'_4_40.p','wb') as f:
        pickle.dump(cv.cv_results_,f)     

In [ ]:
!rm -r ./MI_EEG_ClassMeth
!rm -r ./__MACOSX
!rm ./MI_EEG_ClassMeth.zip